<a href="https://colab.research.google.com/github/DrAlexSanz/YOLO-test/blob/master/YOLO_test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements a simple object detection model. This is greatly inspired on the link here: [Awesome blog post!](https://towardsdatascience.com/yolov2-to-detect-your-own-objects-soccer-ball-using-darkflow-a4f98d5ce5bf)

So far this is the best tutorial I have found. Many people implement this for a given system (linux, ios...) or with a video feed. This is ok, but I want to focus on what I want to do, object detection. Next step is a mask/semantic segmentation and on a much longer time scale, caption generation.

Another useful link, stanford course presentation about this topic: [Slides](http://cs231n.stanford.edu/slides/2018/cs231n_2018_ds06.pdf)



In [0]:
import numpy as np
import keras
import tensorflow as tf

import shutil
import glob
import os # for chdir

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [18, 12]
import h5py

from PIL import Image

from keras import layers, optimizers
from keras.layers import Input, Dense, Conv2D, Activation, ZeroPadding2D, BatchNormalization, Flatten, Add
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.models import Model, Sequential

from keras.preprocessing import image

from keras.initializers import glorot_uniform

from keras.utils import layer_utils, plot_model, to_categorical

from keras.callbacks import History, ModelCheckpoint

from keras.models import load_model

keras.backend.set_image_data_format('channels_last')

%matplotlib inline

import cv2

print("Everything imported correctly")

Everything imported correctly


Now mount the files and then clone the git of the keras implementantion of the yolo model. Then download the weights  of the original model. It takes about 7 minutes, careful with deleting them.

In [0]:
#First remove the folder and everything it contains.

# !rm -rf YOLO-test
!rm -rf keras-yolo3

#Now I mount the folder

from google.colab import drive

drive.mount("/content/gdrive/")
root_path = "gdrive/My Drive/YOLO/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
# !cd gdrive/My Drive/YOLO/
!rm -rf keras-yolo3

!git clone https://github.com/qqwweee/keras-yolo3.git
!cd keras-yolo3

# !wget https://pjreddie.com/media/files/yolov3.weights

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 150.95 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (65/65), done.


Check in which directory I am. It's been a trillion years since I last used the linux commands, I'll keep some comments for the future.

In [0]:
%pwd

'/content'

In [0]:
shutil.copy("/content/gdrive/My Drive/YOLO/yolov3_custom.cfg", "/content/keras-yolo3")

'/content/keras-yolo3/yolov3_custom.cfg'

Now that I am in the correct directory I need to convert the weights I downloaded from darknet to keras. It takes some time but it's provided in the github of the keras implementation.

The syntax is: !python, then file you want to convert. The config file with the weights file, which will be saved at the location of the last argument.

In [0]:
%pwd

'/content'

In [0]:
%cd "/content/keras-yolo3"

/content/keras-yolo3


In [0]:
!python convert.py "yolov3_custom.cfg" "../yolov3.weights" "../yolo.h5"

Now I will load the model

Now I should make sure I am in the correct directory

In [0]:
!pwd

/content/keras-yolo3


I'm not, so I change with !cd

In [0]:
%cd /content

/content


First I am going to load the train and test samples and then I start with the model.

First is to call X and Y to the pics and annotations.

In [0]:
pics = []
picnames = glob.glob("/content/gdrive/My Drive/YOLO/images/*.png")
picnames.sort()

# About 100 times faster than a loop, list comprehensions!! I already knew, but this is still incredible.

X = [Image.open(img) for img in picnames]

#Comprehension lists are very confusing, sotranslating into a for loop it's:

# for name in picnames:
#     img = Image.open(name)
#     pics.append(img)


annotnames = glob.glob("/content/gdrive/My Drive/YOLO/annotations/*.xml")
annotnames.sort()

Y = [an for an in annotnames]



Check that the lengths are ok

In [0]:
# picnames
len(X)


195

In [0]:
loaded_model = load_model("yolo.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


And the model was not compiled, add the weights and compile manually. First let's plot it to check that everything looks correct, which it does.

In [0]:
plot_model(loaded_model, show_shapes = True, to_file = "loaded_model.png")

In [0]:
# loaded_model.load_weights("yolov3.weights")

opti = optimizers.Adam(lr = 1e-3)

checkpointer = ModelCheckpoint(filepath = "best_weights.h5", monitor = "val_acc", save_best_only = True)

loaded_model.compile(loss = "categorical_crossentropy", optimizer = opti, metrics = ["accuracy"])
hist = History()



In [0]:
num_epochs = 10

loaded_model.fit(train_x, train_y, epochs = num_epochs, batch_size = 32, validation_data = (test_x, test_y), shuffle = True, callbacks = [hist, checkpointer])

# loaded_model.load_weights("Resnet/best_weights.h5")

In [0]:
#Maybe I use this later. Just in case I leave it here to avoid copying it later

#Now read data

# with h5py.File("Resnet/test_signs.h5") as test:
#   test_x_1 = np.array(test["test_set_x"][:]) # your train set features
#   test_y_1 = np.array(test["test_set_y"][:]) # your train set labels
#   classes = np.array(test["list_classes"][:]) # the list of classes

# with h5py.File("Resnet/train_signs.h5") as train:
#   train_x_1 = np.array(train["train_set_x"][:]) # train set features
#   train_y_1 = np.array(train["train_set_y"][:]) # train set labels

# print("Everything loaded and imported")